In [1]:
include("utils.jl")
include("EventDataSets.jl")
include("DataTypes.jl")
using DataFrames, Dates, CSV, FileIO, JLD2, DataFramesMeta, Chain,  MixedModels, Random, StatsBase, StatsModels,  DisplayAs

event_spectra = load("/Users/leobreston/Documents/Julia/ROIexp/Results/event_spectra.jld2", "event_spectra")

fbands = (respiratory = [3, 12], theta=[5,10], beta = [15, 35], gamma_low = [50, 59], gamma_high = [70, 100])

for b in keys(fbands)
    @transform!(event_spectra, @byrow $(b) = getbands(:specbands,String(b)).data[1])       
end

eventspecbands=@select(event_spectra, :rat, :behavior_type, :agent_type, :region, $(keys(fbands)...))

,rat,behavior_type,agent_type,region,respiratory,theta,beta
,String7,Abstrac…,String7,String?,Float64,Float64,Float64
1,RRSD17,Approach_Rat,Object,amyg,0.0398035,0.0514335,0.00505864
2,RRSD17,Approach_Rat,Object,mob,0.0633624,0.0972202,0.00484133
3,RRSD17,Approach_Rat,Object,amyg,0.0804163,0.139151,0.00470237
4,RRSD17,Approach_Rat,Object,mob,0.0802137,0.0853531,0.00534392
5,RRSD17,Approach_Rat,Object,ca2,0.10177,0.175585,0.00373058
6,RRSD17,Approach_Rat,Object,amyg,0.0271242,0.0389647,0.0051961
7,RRSD17,Approach_Rat,Object,mob,0.0628778,0.0829426,0.00558754
8,RRSD17,Approach_Rat,Object,ca2,0.0442515,0.0716124,0.00458202
9,RRSD17,Approach_Rat,Object,amyg,0.0166727,0.0269926,0.00382015


Hypothesis 1. It is hypothesized that the robot’s approach/retreat will result in an increase in amplitude of the MOB respiratory oscillation, due to an autonomic shift in alertness. We should still expect a theta rhythm in the rat, because they may or may not be exploring during that time.

In [2]:
hyposub=@subset(eventspecbands, :behavior_type .∈ Ref(["null","Approach_Robot","Retreat_Robot"]), :region.=="mob")
fm=@formula(respiratory ~ 1 + behavior_type*1 + (1|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:behavior_type => DummyCoding(;base="null")))

Minimizing 2 	 Time: 0:00:00 ( 0.19  s/it)
  objective:  -2351.5570055434177

Minimizing 26 	 Time: 0:00:00 (23.89 ms/it)


|                               |    Est. |     SE |     z |      p |  σ_rat |
|:----------------------------- | -------:| ------:| -----:| ------:| ------:|
| (Intercept)                   |  0.0462 | 0.0056 |  8.20 | <1e-15 | 0.0090 |
| behavior_type: Approach_Robot | -0.0002 | 0.0050 | -0.05 | 0.9619 |        |
| behavior_type: Retreat_Robot  | -0.0015 | 0.0028 | -0.53 | 0.5940 |        |
| Residual                      |  0.0179 |        |       |        |        |


Hypothesis 2. It is also hypothesized that hippocampal theta oscillation will increase in amplitude during rearing and the rat’s approaches and retreats in each condition, due to theta being an exploration related sensorimotor rhythm. This difference should also be present in amygdala (though may be due to volume conduction.

In [3]:
hyposub=@subset(eventspecbands, :behavior_type .∈ Ref(["null","Approach_Rat","Retreat_Rat"]), :region.∈Ref(["ca2","amyg"]))
fm=@formula(theta ~0+region+ behavior_type&region + zerocorr(0+region|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:behavior_type => DummyCoding(;base="null")))

Minimizing 2 	 Time: 0:00:00 ( 0.42  s/it)
  objective:  -6983.321070979172

Minimizing 29 	 Time: 0:00:00 (28.72 ms/it)


Minimizing 30 	 Time: 0:00:00 (27.77 ms/it)
  objective:  -6989.936132411475


|                                            |    Est. |     SE |     z |      p |  σ_rat |
|:------------------------------------------ | -------:| ------:| -----:| ------:| ------:|
| region: amyg                               |  0.0155 | 0.0116 |  1.34 | 0.1796 | 0.0194 |
| region: ca2                                |  0.0763 | 0.0017 | 44.50 | <1e-99 | 0.0000 |
| behavior_type: Approach_Rat & region: amyg | -0.0071 | 0.0040 | -1.77 | 0.0763 |        |
| behavior_type: Retreat_Rat & region: amyg  |  0.0041 | 0.0019 |  2.12 | 0.0337 |        |
| behavior_type: Approach_Rat & region: ca2  | -0.0087 | 0.0043 | -2.04 | 0.0411 |        |
| behavior_type: Retreat_Rat & region: ca2   |  0.0042 | 0.0023 |  1.84 | 0.0661 |        |
| Residual                                   |  0.0284 |        |       |        |        |


Hypothesis 3. We expect that beta rhythms will be present when the animal is moving (rearing, approach/retreat), and not present during immobility.

In [4]:
hyposub=@subset(eventspecbands, :behavior_type .∈ Ref(["null","Immobility","Rearing","Approach_Rat","Retreat_Rat"]))
fm=@formula(beta ~0+region+ behavior_type&region + zerocorr(0+region|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:behavior_type => DummyCoding(;base="null")))


|                                            |    Est. |     SE |     z |      p |  σ_rat |
|:------------------------------------------ | -------:| ------:| -----:| ------:| ------:|
| region: amyg                               |  0.0015 | 0.0012 |  1.26 | 0.2078 | 0.0021 |
| region: ca2                                |  0.0038 | 0.0003 | 13.51 | <1e-40 | 0.0004 |
| region: mob                                |  0.0046 | 0.0001 | 46.09 | <1e-99 | 0.0000 |
| behavior_type: Approach_Rat & region: amyg |  0.0008 | 0.0003 |  2.83 | 0.0046 |        |
| behavior_type: Immobility & region: amyg   |  0.0004 | 0.0002 |  1.80 | 0.0726 |        |
| behavior_type: Rearing & region: amyg      |  0.0006 | 0.0002 |  2.75 | 0.0059 |        |
| behavior_type: Retreat_Rat & region: amyg  |  0.0001 | 0.0001 |  0.69 | 0.4882 |        |
| behavior_type: Approach_Rat & region: ca2  |  0.0005 | 0.0003 |  1.63 | 0.1021 |        |
| behavior_type: Immobility & region: ca2    |  0.0005 | 0.0002 |  2.35 | 0.0190 |        |
| behavior_type: Rearing & region: ca2       |  0.0005 | 0.0003 |  2.09 | 0.0369 |        |
| behavior_type: Retreat_Rat & region: ca2   | -0.0002 | 0.0002 | -1.55 | 0.1202 |        |
| behavior_type: Approach_Rat & region: mob  |  0.0007 | 0.0003 |  2.52 | 0.0118 |        |
| behavior_type: Immobility & region: mob    | -0.0014 | 0.0002 | -6.69 | <1e-10 |        |
| behavior_type: Rearing & region: mob       |  0.0002 | 0.0002 |  0.84 | 0.4025 |        |
| behavior_type: Retreat_Rat & region: mob   |  0.0004 | 0.0001 |  2.66 | 0.0078 |        |
| Residual                                   |  0.0020 |        |       |        |        |


Hypothesis 4. It is hypothesized that theta will show increased amplitude during behaviors associated with exploration like rearing, and approach/avoidance behaviors, when compared with regulatory behaviors like grooming and immobility. This means that rearing will be significant different from immobility/grooming in both respiratory and theta rhythms in MOB, CA2, and Amyg. 

In [5]:
hyposub=@subset(eventspecbands, :behavior_type .∈ Ref(["Immobility","Grooming","Rearing","Approach_Rat","Retreat_Rat"]))
function fun(btype)
    if btype .== "Immobility"
        return "Immobility & Grooming"
    elseif btype .== "Grooming"
        return "Immobility & Grooming"
    else
        return "Rearing & Approach/Retreat"
    end
end
@transform!(hyposub, @byrow :combined_behavior = fun(:behavior_type))
fm=@formula(theta ~0+region+ combined_behavior&region + zerocorr(0+region|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:combined_behavior => DummyCoding(;base="Immobility & Grooming")))

|                                                              |   Est. |     SE |     z |      p |  σ_rat |
|:------------------------------------------------------------ | ------:| ------:| -----:| ------:| ------:|
| region: amyg                                                 | 0.0053 | 0.0114 |  0.47 | 0.6391 | 0.0189 |
| region: ca2                                                  | 0.0615 | 0.0046 | 13.22 | <1e-39 | 0.0056 |
| region: mob                                                  | 0.0398 | 0.0065 |  6.13 | <1e-09 | 0.0097 |
| combined_behavior: Rearing & Approach/Retreat & region: amyg | 0.0135 | 0.0026 |  5.26 | <1e-06 |        |
| combined_behavior: Rearing & Approach/Retreat & region: ca2  | 0.0239 | 0.0029 |  8.23 | <1e-15 |        |
| combined_behavior: Rearing & Approach/Retreat & region: mob  | 0.0216 | 0.0026 |  8.23 | <1e-15 |        |
| Residual                                                     | 0.0306 |        |       |        |        |


In [6]:
fm=@formula(respiratory ~0+region+ combined_behavior&region + zerocorr(0+region|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:combined_behavior => DummyCoding(;base="Immobility & Grooming")))


|                                                              |   Est. |     SE |     z |      p |  σ_rat |
|:------------------------------------------------------------ | ------:| ------:| -----:| ------:| ------:|
| region: amyg                                                 | 0.0131 | 0.0106 |  1.24 | 0.2153 | 0.0180 |
| region: ca2                                                  | 0.0554 | 0.0015 | 36.57 | <1e-99 | 0.0000 |
| region: mob                                                  | 0.0459 | 0.0051 |  9.02 | <1e-18 | 0.0081 |
| combined_behavior: Rearing & Approach/Retreat & region: amyg | 0.0008 | 0.0015 |  0.53 | 0.5943 |        |
| combined_behavior: Rearing & Approach/Retreat & region: ca2  | 0.0031 | 0.0017 |  1.81 | 0.0703 |        |
| combined_behavior: Rearing & Approach/Retreat & region: mob  | 0.0031 | 0.0016 |  1.98 | 0.0473 |        |
| Residual                                                     | 0.0184 |        |       |        |        |


Hypothesis 5. It is hypothesized that robot will exhibit a higher low-gamma rhythm in MOB for the robot condition when compared with the object and rat conditions for immobility and grooming behaviors. 


In [7]:
hyposub=@subset(eventspecbands, :behavior_type .∈ Ref(["Immobility","Grooming"]), :region.=="mob")
fm=@formula(gamma_low ~0+behavior_type+ agent_type&behavior_type + (1|rat))
fm1 = fit(MixedModel,fm, hyposub,contrasts = Dict(:agent_type => DummyCoding(;base="Robot"))) 


|                                                |    Est. |     SE |     z |      p |  σ_rat |
|:---------------------------------------------- | -------:| ------:| -----:| ------:| ------:|
| behavior_type: Grooming                        |  0.0033 | 0.0003 | 11.38 | <1e-29 |        |
| behavior_type: Immobility                      |  0.0021 | 0.0002 | 10.02 | <1e-22 |        |
| agent_type: Object & behavior_type: Grooming   | -0.0016 | 0.0005 | -3.20 | 0.0014 |        |
| agent_type: Rat & behavior_type: Grooming      | -0.0014 | 0.0004 | -3.74 | 0.0002 |        |
| agent_type: Object & behavior_type: Immobility | -0.0005 | 0.0003 | -1.76 | 0.0790 |        |
| agent_type: Rat & behavior_type: Immobility    | -0.0013 | 0.0004 | -3.40 | 0.0007 |        |
| (Intercept)                                    |         |        |       |        | 0.0000 |
| Residual                                       |  0.0013 |        |       |        |        |


Hypothesis 6. It is hypothesized that there will be a larger amplitude low gamma component in MOB when the robot is approaching/retreating.


In [8]:
hyposub=@subset(eventspecbands, :region.=="mob")
fm=@formula(gamma_low ~1+behavior_type + (1|rat))
fm1 = fit(MixedModel,fm, hyposub,contrasts = Dict(:agent_type => DummyCoding(;base="Robot")))

|                               |    Est. |     SE |     z |      p |  σ_rat |
|:----------------------------- | -------:| ------:| -----:| ------:| ------:|
| (Intercept)                   |  0.0022 | 0.0004 |  5.83 | <1e-08 | 0.0006 |
| behavior_type: Approach_Robot |  0.0008 | 0.0004 |  1.97 | 0.0483 |        |
| behavior_type: Grooming       |  0.0009 | 0.0002 |  3.97 | <1e-04 |        |
| behavior_type: Hut            | -0.0006 | 0.0013 | -0.49 | 0.6210 |        |
| behavior_type: Immobility     |  0.0001 | 0.0002 |  0.51 | 0.6082 |        |
| behavior_type: Rearing        | -0.0001 | 0.0002 | -0.61 | 0.5434 |        |
| behavior_type: Retreat_Rat    | -0.0002 | 0.0002 | -0.97 | 0.3330 |        |
| behavior_type: Retreat_Robot  |  0.0012 | 0.0003 |  4.53 | <1e-05 |        |
| behavior_type: null           |  0.0001 | 0.0002 |  0.35 | 0.7252 |        |
| Residual                      |  0.0013 |        |       |        |        |


Hypothesis 7. High gamma amplitude should be highest for the rat condition in the MOB. The robot and object should have similar high gamma. But instead, it appears that the robot and rat high gamma is larger than the object, and there is no discernable difference between rat and robot high gamma in MOB. 

In [9]:
hyposub=@subset(eventspecbands, :behavior_type .!= "null", :region.=="mob")
fm=@formula(gamma_high ~ 1+agent_type + (1|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(;base="Rat")))

|                    |    Est. |     SE |     z |      p |  σ_rat |
|:------------------ | -------:| ------:| -----:| ------:| ------:|
| (Intercept)        |  0.0020 | 0.0002 | 12.34 | <1e-34 | 0.0002 |
| agent_type: Object | -0.0005 | 0.0001 | -5.51 | <1e-07 |        |
| agent_type: Robot  |  0.0001 | 0.0001 |  1.64 | 0.1005 |        |
| Residual           |  0.0010 |        |       |        |        |


Hypothesis 8. We expect a higher amplitude theta rhythm in the CA2 and Amygdala for the rat approach/retreat conditions, compared to the robot and object.

In [10]:
hyposub=@subset(eventspecbands, :behavior_type .∈ Ref(["Approach_Rat","Retreat_Rat","Approach_Robot","Retreat_Robot"]), :region.!="mob")
fm=@formula(theta ~0+behavior_type&region + agent_type&behavior_type&region + zerocorr(0+region|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(;base="Rat")))


┌ Warning: Fixed-effects matrix is rank deficient
└ @ MixedModels /Users/leobreston/.julia/packages/MixedModels/9ktwJ/src/Xymat.jl:41


|                                                                   |    Est. |     SE |     z |      p |  σ_rat |
|:----------------------------------------------------------------- | -------:| ------:| -----:| ------:| ------:|
| behavior_type: Approach_Rat & region: amyg                        |  0.0185 | 0.0130 |  1.43 | 0.1532 |        |
| behavior_type: Approach_Robot & region: amyg                      | -0.0000 |    NaN |   NaN |    NaN |        |
| behavior_type: Retreat_Rat & region: amyg                         |  0.0224 | 0.0121 |  1.85 | 0.0645 |        |
| behavior_type: Retreat_Robot & region: amyg                       |  0.0065 | 0.0126 |  0.51 | 0.6074 |        |
| behavior_type: Approach_Rat & region: ca2                         |  0.0620 | 0.0049 | 12.67 | <1e-36 |        |
| behavior_type: Approach_Robot & region: ca2                       |  0.0523 | 0.0092 |  5.66 | <1e-07 |        |
| behavior_type: Retreat_Rat & region: ca2                          |  0.0815 | 0.0016 | 51.64 | <1e-99 |        |
| behavior_type: Retreat_Robot & region: ca2                        |  0.0621 | 0.0050 | 12.50 | <1e-35 |        |
| agent_type: Object & behavior_type: Approach_Rat & region: amyg   | -0.0000 |    NaN |   NaN |    NaN |        |
| agent_type: Robot & behavior_type: Approach_Rat & region: amyg    | -0.0132 | 0.0077 | -1.71 | 0.0872 |        |
| agent_type: Object & behavior_type: Approach_Robot & region: amyg | -0.0000 |    NaN |   NaN |    NaN |        |
| agent_type: Robot & behavior_type: Approach_Robot & region: amyg  |  0.0119 | 0.0143 |  0.83 | 0.4046 |        |
| agent_type: Object & behavior_type: Retreat_Rat & region: amyg    |  0.0085 | 0.0057 |  1.48 | 0.1387 |        |
| agent_type: Robot & behavior_type: Retreat_Rat & region: amyg     | -0.0161 | 0.0053 | -3.07 | 0.0022 |        |
| agent_type: Object & behavior_type: Retreat_Robot & region: amyg  | -0.0000 |    NaN |   NaN |    NaN |        |
| agent_type: Robot & behavior_type: Retreat_Robot & region: amyg   | -0.0000 |    NaN |   NaN |    NaN |        |
| agent_type: Object & behavior_type: Approach_Rat & region: ca2    |  0.0141 | 0.0078 |  1.81 | 0.0698 |        |
| agent_type: Robot & behavior_type: Approach_Rat & region: ca2     | -0.0000 |    NaN |   NaN |    NaN |        |
| agent_type: Object & behavior_type: Approach_Robot & region: ca2  | -0.0000 |    NaN |   NaN |    NaN |        |
| agent_type: Robot & behavior_type: Approach_Robot & region: ca2   | -0.0000 |    NaN |   NaN |    NaN |        |
| agent_type: Object & behavior_type: Retreat_Rat & region: ca2     |  0.0107 | 0.0076 |  1.41 | 0.1572 |        |
| agent_type: Robot & behavior_type: Retreat_Rat & region: ca2      | -0.0213 | 0.0061 | -3.48 | 0.0005 |        |
| agent_type: Object & behavior_type: Retreat_Robot & region: ca2   | -0.0000 |    NaN |   NaN |    NaN |        |
| agent_type: Robot & behavior_type: Retreat_Robot & region: ca2    | -0.0000 |    NaN |   NaN |    NaN |        |
| region: ca2                                                       |         |        |       |        | 0.0000 |
| region: amyg                                                      |         |        |       |        | 0.0195 |
| Residual                                                          |  0.0277 |        |       |        |        |


Hypothesis 9. It is hypothesized that the amygdala beta rhythm shows a difference between grooming/immobility and rearing. It would be expected that immobility shows the least beta, with grooming showing some due to arm movements, and rearing showing the most. 

In [11]:
hyposub=@subset(eventspecbands, :behavior_type .∈ Ref(["Immobility","Grooming","Rearing"]), :region.=="amyg")
fm=@formula(beta ~1+behavior_type + (1|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:behavior_type => DummyCoding(;base="Immobility")))


|                         |    Est. |     SE |     z |      p |  σ_rat |
|:----------------------- | -------:| ------:| -----:| ------:| ------:|
| (Intercept)             |  0.0018 | 0.0013 |  1.41 | 0.1589 | 0.0022 |
| behavior_type: Grooming | -0.0007 | 0.0003 | -2.51 | 0.0122 |        |
| behavior_type: Rearing  |  0.0002 | 0.0003 |  0.87 | 0.3824 |        |
| Residual                |  0.0019 |        |       |        |        |


Hypothesis 10. For regulatory behaviors (grooming/immobility) there should be a shift to lower frequencies within the respiratory range and more low gamma, while exploratory behaviors should shift to theta frequency and high gamma (rearing, approach/retreat). 


In [12]:
hyposub=@subset(eventspecbands, :behavior_type .∈ Ref(["Immobility","Grooming","Rearing","Approach_Rat","Retreat_Rat"]))
function fun(btype)
    if btype .== "Immobility"
        return "Immobility & Grooming"
    elseif btype .== "Grooming"
        return "Immobility & Grooming"
    else
        return "Rearing & Approach/Retreat"
    end
end
@transform!(hyposub, @byrow :combined_behavior = fun(:behavior_type))

fm=@formula(respiratory ~0+region+ combined_behavior&region + zerocorr(0+region|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:combined_behavior => DummyCoding(;base="Immobility & Grooming")))

|                                                              |   Est. |     SE |     z |      p |  σ_rat |
|:------------------------------------------------------------ | ------:| ------:| -----:| ------:| ------:|
| region: amyg                                                 | 0.0131 | 0.0106 |  1.24 | 0.2153 | 0.0180 |
| region: ca2                                                  | 0.0554 | 0.0015 | 36.57 | <1e-99 | 0.0000 |
| region: mob                                                  | 0.0459 | 0.0051 |  9.02 | <1e-18 | 0.0081 |
| combined_behavior: Rearing & Approach/Retreat & region: amyg | 0.0008 | 0.0015 |  0.53 | 0.5943 |        |
| combined_behavior: Rearing & Approach/Retreat & region: ca2  | 0.0031 | 0.0017 |  1.81 | 0.0703 |        |
| combined_behavior: Rearing & Approach/Retreat & region: mob  | 0.0031 | 0.0016 |  1.98 | 0.0473 |        |
| Residual                                                     | 0.0184 |        |       |        |        |


In [13]:
fm=@formula(theta ~0+region+ combined_behavior&region + zerocorr(0+region|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:combined_behavior => DummyCoding(;base="Immobility & Grooming")))

|                                                              |   Est. |     SE |     z |      p |  σ_rat |
|:------------------------------------------------------------ | ------:| ------:| -----:| ------:| ------:|
| region: amyg                                                 | 0.0053 | 0.0114 |  0.47 | 0.6391 | 0.0189 |
| region: ca2                                                  | 0.0615 | 0.0046 | 13.22 | <1e-39 | 0.0056 |
| region: mob                                                  | 0.0398 | 0.0065 |  6.13 | <1e-09 | 0.0097 |
| combined_behavior: Rearing & Approach/Retreat & region: amyg | 0.0135 | 0.0026 |  5.26 | <1e-06 |        |
| combined_behavior: Rearing & Approach/Retreat & region: ca2  | 0.0239 | 0.0029 |  8.23 | <1e-15 |        |
| combined_behavior: Rearing & Approach/Retreat & region: mob  | 0.0216 | 0.0026 |  8.23 | <1e-15 |        |
| Residual                                                     | 0.0306 |        |       |        |        |


In [14]:

fm=@formula(gamma_low ~0+region+ combined_behavior&region + zerocorr(0+region|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:combined_behavior => DummyCoding(;base="Immobility & Grooming")))

|                                                              |    Est. |     SE |     z |      p |  σ_rat |
|:------------------------------------------------------------ | -------:| ------:| -----:| ------:| ------:|
| region: amyg                                                 |  0.0005 | 0.0003 |  1.85 | 0.0636 | 0.0005 |
| region: ca2                                                  |  0.0010 | 0.0001 | 15.06 | <1e-50 | 0.0000 |
| region: mob                                                  |  0.0026 | 0.0004 |  7.06 | <1e-11 | 0.0006 |
| combined_behavior: Rearing & Approach/Retreat & region: amyg |  0.0002 | 0.0001 |  3.24 | 0.0012 |        |
| combined_behavior: Rearing & Approach/Retreat & region: ca2  |  0.0001 | 0.0001 |  2.00 | 0.0454 |        |
| combined_behavior: Rearing & Approach/Retreat & region: mob  | -0.0006 | 0.0001 | -8.59 | <1e-17 |        |
| Residual                                                     |  0.0008 |        |       |        |        |


In [15]:
fm=@formula(gamma_high ~0+region+ combined_behavior&region + zerocorr(0+region|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:combined_behavior => DummyCoding(;base="Immobility & Grooming")))

|                                                              |   Est. |     SE |     z |      p |  σ_rat |
|:------------------------------------------------------------ | ------:| ------:| -----:| ------:| ------:|
| region: amyg                                                 | 0.0004 | 0.0002 |  1.54 | 0.1239 | 0.0004 |
| region: ca2                                                  | 0.0005 | 0.0001 |  8.36 | <1e-16 | 0.0000 |
| region: mob                                                  | 0.0016 | 0.0001 | 30.10 | <1e-99 | 0.0000 |
| combined_behavior: Rearing & Approach/Retreat & region: amyg | 0.0004 | 0.0001 |  6.67 | <1e-10 |        |
| combined_behavior: Rearing & Approach/Retreat & region: ca2  | 0.0003 | 0.0001 |  3.80 | 0.0001 |        |
| combined_behavior: Rearing & Approach/Retreat & region: mob  | 0.0002 | 0.0001 |  3.47 | 0.0005 |        |
| Residual                                                     | 0.0007 |        |       |        |        |



Hypothesis 11. Grooming during OpenField should have a high low gamma in MOB due to stress during immobility and grooming, in comparison with the rat/robot/object. It is interesting to examine the robot's low gamma in MOB relative to the OpenField. 


Hypothesis 12. It is also possible that the OpenField trials are high exploratory, and thus have a small low gamma component. We see in the behavioral data that rearing occurs way more often during the open field trials, which suggests an exploratory regime. 

Hypothesis 13. It is hypothesized that there will be a difference between the robot and object conditions in the low gamma range of the amygdala during immobility behaviors, with the robot eliciting a higher low gamma component. 

In [16]:
hyposub=@subset(eventspecbands, :behavior_type.=="Immobility", :region.=="amyg")
fm=@formula(gamma_low ~1+agent_type + (1|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(;base="Robot")))


|                    |    Est. |     SE |     z |      p |  σ_rat |
|:------------------ | -------:| ------:| -----:| ------:| ------:|
| (Intercept)        |  0.0006 | 0.0001 |  3.84 | 0.0001 | 0.0002 |
| agent_type: Object | -0.0000 | 0.0001 | -0.37 | 0.7090 |        |
| agent_type: Rat    |  0.0005 | 0.0001 |  4.20 | <1e-04 |        |
| Residual           |  0.0004 |        |       |        |        |
